In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
import holoviews as hv
import hvplot.pandas  # noqa
from functools import cache
from constants import *
from bokeh.plotting import figure, show, output_notebook
output_notebook()


test_date = dt.datetime.strptime("2024-01-01", "%Y-%m-%d")
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE

# Get nifty candles for minute
# For first minute, get premium strike price for call and puts which is price at ~30 
# Draw them here 
# Try to match the pattern 

/home/nitin/nse-reports-trade/utils.py:28: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  scrip_df = pd.read_csv(file_path)


Loading BokehJS ...

In [2]:
nifty_candles = ut.get_data(symbol=SYMBOL, date=test_date, interval=INTERVAL, exchange=EXCHANGE)
atm_strike = 21700
otm_call_strike = 22100
otm_put_strike = 21300
expiry = ut.find_nclosest_expiry(SYMBOL, test_date, 1)

call_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=expiry, cur_dt=test_date.date(), strike_price=otm_call_strike, option_type=OPTION_TYPE_CALL)
put_df = ic.get_opt_pre_df(symbol=IC_SYMBOL, expiry=expiry, cur_dt=test_date.date(), strike_price=otm_put_strike, option_type=OPTION_TYPE_PUT)

In [105]:
# Create a vbar chart
# Create a box plot 
# Merge them together and come up with a good candlestick chart
from bokeh.models import ColumnDataSource, BooleanFilter, CDSView, IndexFilter, CrosshairTool, HoverTool, WheelZoomTool, DatetimeTickFormatter, DatetimeTicker, TickFormatter, CustomJSTickFormatter, Label, CustomJS
from bokeh.layouts import gridplot
from bokeh.io import curdoc
from bokeh.themes import Theme


# def create_candlestick_plot(df):
df = call_df
cds = ColumnDataSource(df)
green = CDSView(filter=BooleanFilter(df['close'] >= df['open']))
red = CDSView(filter=BooleanFilter(df['close'] < df['open']))
w = 60*1000
p = figure(
    x_axis_type="datetime",
    title=f"Minute Candles",
    min_width=1000,
    min_height=700,
    background_fill_color="#1e1e1e"
)

# Segments for high-low
p.segment(x0='date_time', y0='high', x1='date_time', y1='low', color="#26a69a", source=cds, view=green)
p.segment(x0='date_time', y0='high', x1='date_time', y1='low', color="#ef5350", source=cds, view=red)

# Bars for open-close
p.vbar(x='date_time', width=w, top='open', bottom='close', fill_color="#26a69a", line_color="black", source=cds, view=green)
p.vbar(x='date_time', width=w, top='close', bottom='open', fill_color="#ef5350", line_color="black", source=cds, view=red)

# p.xaxis.formatter = DatetimeTickFormatter(
#     minutes="%H:%M",
#     hours="%H:%M",
#     days="%H:%M",
#     months="%H:%M",
#     years="%H:%M"
# )

p.xaxis.formatter = CustomJSTickFormatter(code="""
    var date = new Date(tick);
    var hours = date.getUTCHours();
    var minutes = date.getUTCMinutes();
    var suffix = (hours >= 12) ? 'PM' : 'AM';
    hours = (hours % 12) || 12;
    minutes = minutes < 10 ? '0' + minutes : minutes;
    return hours + ':' + minutes;
""")

p.xaxis.ticker = DatetimeTicker(
    desired_num_ticks=30,
    num_minor_ticks=5
)

p.xaxis.major_label_orientation = 0.5
p.grid.grid_line_alpha = 0.3

crosshair_tool = CrosshairTool(
    dimensions="both",
    line_color="red",
    line_alpha=0.8,
)
p.add_tools(crosshair_tool)
hover = HoverTool(tooltips=[("Date", "@date_time{%H:%M}"), ("Open", "@open"), ("High", "@high"), ("Low", "@low"), ("Close", "@close")], formatters={'@date_time': 'datetime'})
p.add_tools(hover)
wheel_zoom = WheelZoomTool()
p.add_tools(wheel_zoom)
p.toolbar.active_scroll = wheel_zoom


tradingview_theme = Theme(json={
    'attrs': {
        'figure': {
            'background_fill_color': "#1e1e1e",
            'border_fill_color': "#1e1e1e",
            'outline_line_color': "#393939"
        },
        'Axis': {
            'major_label_text_color': "#e0e0e0",
            'axis_label_text_color': "#e0e0e0",
            'major_tick_line_color': "#393939",
            'minor_tick_line_color': "#393939",
            'axis_line_color': "#393939"
        },
        'Grid': {
            'grid_line_color': "#393939"
        }
    }
})


# Create labels for crosshair values on the axes
x_label = Label(x=0, y=0, x_units='data', y_units='screen', text='', text_color='white',
                text_font_size='10pt', background_fill_color='#1e1e1e',
                background_fill_alpha=0.8, text_align='left', text_baseline='bottom')

y_label = Label(x=0, y=0, x_units='screen', y_units='data', text='', text_color='white',
                text_font_size='10pt', background_fill_color='#1e1e1e',
                background_fill_alpha=0.8, text_align='left', text_baseline='bottom')

p.add_layout(x_label, 'below')
p.add_layout(y_label, 'below')

# CustomJS callback to update labels
callback = CustomJS(args={'x_label': x_label, 'y_label': y_label, 'plot': p}, code="""
    const { x, y } = cb_data['geometry'];
    const { sx, sy } = cb_data['geometry'];
    const plotHeight = plot.height;
    if (sx !== undefined || sy !== undefined) {
        const date = new Date(x);
        var hours = date.getUTCHours();
        var minutes = date.getUTCMinutes();
        hours = (hours % 12) || 12;
        minutes = minutes < 10 ? '0' + minutes : minutes;
        const xval = hours + ':' + minutes;

        const yValue = y.toFixed(2);

        x_label.x =  x;
        x_label.y = 0;  // Slightly offset from the bottom
        x_label.text = xval;

        y_label.x = 0;  // Slightly offset from the left
        y_label.y = y;
        y_label.text = yValue;

        x_label.visible = true;
        y_label.visible = true;
    }
""")

# Add hover tool to update labels
hover = HoverTool(tooltips=None)

hover.callback = callback

p.add_tools(hover)

curdoc().theme = tradingview_theme


show(p)